In [1]:
import strawberryfields as sf
import tensorflow as tf
import random

In [2]:
tf_displacement_magnitude = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 500

for step in range(steps):
    if random.random() > 0.5:
        mult = 1
    else:
        mult = -1
        
    alpha = 0.7 * mult
        
    eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 7})
    circuit = sf.Program(1)

    displacement_magnitude = circuit.params("displacement_magnitude")

    with circuit.context as q:
        sf.ops.Dgate(alpha, 0.0) | q[0]
        sf.ops.Dgate(displacement_magnitude, 0.0) | q[0]
        
    with tf.GradientTape() as tape:
        results = eng.run(circuit, args={"displacement_magnitude": tf_displacement_magnitude})
        
        # get the probability of fock state |1>
        p_zero = results.state.fock_prob([0])
        p_one = 1 - p_zero
        
        if mult == 1:
            loss = -p_one
        else:
            loss = -p_zero
        
    gradients = tape.gradient(loss, [tf_displacement_magnitude])
    opt.apply_gradients(zip(gradients, [tf_displacement_magnitude]))
    
    if (step + 1) % 10 == 0:
        print("Learned displacement value at step {}: {}".format(step+1, tf_displacement_magnitude.numpy()))

2021-09-22 21:10:07.019728: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Learned displacement value at step 10: 0.1997651606798172
Learned displacement value at step 20: 0.2982299029827118
Learned displacement value at step 30: 0.3937018811702728
Learned displacement value at step 40: 0.48198819160461426
Learned displacement value at step 50: 0.5610660910606384
Learned displacement value at step 60: 0.6287330389022827
Learned displacement value at step 70: 0.6860527396202087
Learned displacement value at step 80: 0.7300508618354797
Learned displacement value at step 90: 0.7674557566642761
Learned displacement value at step 100: 0.7883326411247253
Learned displacement value at step 110: 0.8196666240692139
Learned displacement value at step 120: 0.8507879972457886
Learned displacement value at step 130: 0.8730281591415405
Learned displacement value at step 140: 0.8753596544265747
Learned displacement value at step 150: 0.8815778493881226
Learned displacement value at step 160: 0.8705160617828369
Learned displacement value at step 170: 0.8570134043693542
Learn